In [1]:
import pandas as pd
import numpy as np
import glob
import os
import arcpy
from io import StringIO
from arcgis.gis import GIS

# Python Standard Library Modules
from pathlib import Path
from zipfile import ZipFile

In [2]:
import importlib.util
spec = importlib.util.spec_from_file_location("dbconnect", "G:/My Drive/Python/dbconnect.py")
dbconnect = importlib.util.module_from_spec(spec)
spec.loader.exec_module(dbconnect)

In [3]:
import requests

url = 'https://services.arcgis.com/ZzrwjTRez6FJiOq4/arcgis/rest/services/SamplingActivitiesTable/FeatureServer/1/query?'
kwargs = {}
kwargs['where'] = 'ActivityID is NOT NULL'
kwargs['returnIdsOnly'] = 'false'
kwargs['returnUniqueIdsOnly'] = 'false'
kwargs['returnCountOnly'] ='false'
kwargs['returnDistinctValues'] ='true'
kwargs['outFields'] = '*'
kwargs['orderByFields'] ='ActivityID'
kwargs['sqlFormat'] ='standard'
kwargs['f'] ='pjson'
kwargs['token'] = 'twt1PeCt9UqHmMEe4hceWzSLu6FllZQoVAi3pdSTvTkH9OZBJVv1RveHS6Xzzcaj4kZ3QozNIniFitdBQ2LKU7Bwjz-WNGdP2yVvEXPc7NRJuj0FsV2njhebPV-5nktDAbdcHKwqUdVQQQ-EnW63gHG17KVZJiMjeyJXy2oduZfPPsGzbhfURZGFuaRRCg3WCT-GI8Tr04lnAHBbYT5g1akFQYmf6QV1lAOVVv4DOnM.'
repsns = requests.get(url, params=kwargs)
onlineacts = pd.DataFrame([i['attributes'] for i in repsns.json()['features']])
onlineacts.head()

,ActivityID,ActivityStartDate,ActivityStartTime,GlobalID,JoinField,MonitoringLocationID,OBJECTID,ProjectID,SampleCollEquipmentName,SampleCollMethodID,conduct_uscm,notes,pH,personnel,temperature_c
0,1406012-01,1402272000000,09:20,8f45e0b3-37d7-4247-8585-5110ce44e787,1406012-01-FM,384113109391001,1,UNGWMN,Water Bottle,GRAB,1680.0,None,8.10,None,20.1
1,1406012-03,1402272000000,13:05,3fcfc3c3-0755-4ba2-8b57-958b26698ca4,1406012-03-FM,383210109285801,2,UNGWMN,Water Bottle,GRAB,280.0,None,8.27,None,15.9
2,1406012-04,1402272000000,15:20,2b6e5214-fc01-4c31-89e3-480063cb3bf6,1406012-04-FM,383453109200601,3,UNGWMN,Water Bottle,GRAB,320.0,None,7.97,None,11.5
3,1406012-05,1402272000000,17:00,a7396c4a-0b4c-459e-bf26-03795384ab75,1406012-05-FM,383854109242901,4,UNGWMN,Water Bottle,GRAB,1300.0,None,8.10,None,17.0
4,1406012-06,1402358400000,13:15,4a39ac1b-3765-4a6c-bd45-b22b4578e608,1406012-06-FM,373702109284401,5,UNGWMN,Water Bottle,GRAB,1410.0,None,8.40,None,16.3


In [ ]:
'1805005-10RE1','UNGWMN',404724111562501,'2018-05-08','10:00',NULL,NULL,'paulinkenbrandt','2019-04-15 16:55:42.273037','paulinkenbrandt','2019-04-15 16:55:42.274040',,13063

Import the `wqxsde` module (if it is not installed)

In [4]:
wqxpath = "G:/My Drive/Python/Pycharm/wqxsde/"
sys.path.append(wqxpath)
import wqxsde

assign the relevant paths to the 

In [5]:
sde = wqxsde.GetPaths()
arcpy.env.workspace = sde.enviro
wqx_results_filename = "ResultsExport.xlsx"
wqx_stations_filename = "LocationsExport.xlsx"

# Stations

Compare station export from the CDX to the data in the ArcGIS SDE table.

In [ ]:
wqx_stat_table, sde_stat_table = wqxsde.compare_sde_wqx(wqx_stations_filename,
                                                        sde.enviro, 
                                                        sde.stations_table,
                                                        table_type="stat")

Get data from SDE and prepare for WQX import configuration number 6441 <br>
https://cdx.epa.gov/WQXWeb/ImportConfigurationDetail.aspx?mode=import&impcfg_uid=6441

In [ ]:
stats = wqxsde.SDEStationstoWQX(sde_stat_table,"G:/My Drive/Python/Pycharm/wqxsde")
stats.save_file() # save file to csv
sde_stats = stats.sde_stat_import

# Chemistry

## EPA Lab Data to SDE

In [6]:
epa_file = "G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/EPAdata_results/2018/1807003 FINAL EPA Excel 12 Sep 18 1357.xls"
schema_file_path = "G:/My Drive/Python/Pycharm/wqxsde/examples/Domains.xlsx"
save_path = "G:/My Drive/WORK/Groundwater Chemistry/"

for file in glob.glob("G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/EPAdata_results/2018/*.xls"):

    epa = wqxsde.ProcessEPASheet(file, save_path, schema_file_path)

    epa.append_data()

0 to 13 complete
Transaction started...
INSERT INTO UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities(ActivityID,ProjectID,MonitoringLocationID,ActivityStartDate,ActivityStartTime,notes,personnel,created_user,created_date,last_edited_user,last_edited_date,OBJECTID)
VALUES ('1805005-04RE1','UNGWMN',373702109284401,'2018-05-08','09:45',NULL,NULL,'paulinkenbrandt','2019-04-15 16:55:42.273037','paulinkenbrandt','2019-04-15 16:55:42.274040',,13051),('1805005-04RE1','UNGWMN',373702109284401,'2018-05-08','09:45',NULL,NULL,'paulinkenbrandt','2019-04-15 16:55:42.273037','paulinkenbrandt','2019-04-15 16:55:42.274040',,13052),('1805005-02RE1','UNGWMN',391020111421301,'2018-05-07','13:00',NULL,NULL,'paulinkenbrandt','2019-04-15 16:55:42.273037','paulinkenbrandt','2019-04-15 16:55:42.274040',,13053),('1805005-01RE1','UNGWMN',391159111543401,'2018-05-07','12:20',NULL,NULL,'paulinkenbrandt','2019-04-15 16:55:42.273037','paulinkenbrandt','2019-04-15 16:55:42.274040',,13054),('1805005-03RE1','UNG

Transaction started...
INSERT INTO UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results(ActivityID,MonitoringLocationID,ResultAnalyticalMethodContext,ResultAnalyticalMethodID,ResultSampleFraction,resultvalue,DetecQuantLimitMeasure,ResultDetecQuantLimitUnit,ResultUnit,AnalysisStartDate,ResultDetecQuantLimitType,ResultDetectionCondition,CharacteristicName,MethodSpeciation,characteristicgroup,inwqx,created_user,last_edited_user,created_date,last_edited_date,resultid,OBJECTID)
VALUES ('1806003-11RE1',370820113293501,'USEPA','300.0','Total',419.0,25.0,'mg/l','mg/l','2018-06-29',NULL,NULL,'Sulfate','as SO4','Inorganics, Major, Non-metals',0,'paulinkenbrandt','paulinkenbrandt','2019-04-15 16:56:03.442087','2019-04-15 16:56:03.442588','1806003-11RE1-Sulfate',,14051),('1806003-08RE1',374005113081801,'USEPA','300.0','Total',497.0,25.0,'mg/l','mg/l','2018-06-29',NULL,NULL,'Sulfate','as SO4','Inorganics, Major, Non-metals',0,'paulinkenbrandt','paulinkenbrandt','2019-04-15 16:56:03.442087','2019-04

0 to 286 complete
Transaction started...
INSERT INTO UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results(ActivityID,MonitoringLocationID,ResultAnalyticalMethodContext,ResultAnalyticalMethodID,ResultSampleFraction,resultvalue,DetecQuantLimitMeasure,ResultDetecQuantLimitUnit,ResultUnit,AnalysisStartDate,ResultDetecQuantLimitType,ResultDetectionCondition,CharacteristicName,MethodSpeciation,characteristicgroup,inwqx,created_user,last_edited_user,created_date,last_edited_date,resultid,OBJECTID)
VALUES ('1807003-01',390129110085301,'USEPA','300.0','Total',NULL,0.5,'mg/l','mg/l','2018-07-13','Lower Reporting Limit','Below Reporting Limit','Bromide',NULL,'Inorganics, Major, Non-metals',0,'paulinkenbrandt','paulinkenbrandt','2019-04-15 16:56:26.833401','2019-04-15 16:56:26.834404','1807003-01-Bromide',,14051),('1807003-01',390129110085301,'USEPA','300.0','Total',26.0,1.0,'mg/l','mg/l','2018-07-13',NULL,NULL,'Chloride',NULL,'Inorganics, Major, Non-metals',0,'paulinkenbrandt','paulinkenbrandt','

success!
0 to 234 complete
Transaction started...
INSERT INTO UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities(ActivityID,ProjectID,MonitoringLocationID,ActivityStartDate,ActivityStartTime,notes,personnel,created_user,created_date,last_edited_user,last_edited_date,OBJECTID)
VALUES ('1808006-03','UNGWMN',385630112321701,'2018-08-06','15:50',NULL,NULL,'paulinkenbrandt','2019-04-15 16:56:52.228920','paulinkenbrandt','2019-04-15 16:56:52.229422',,13051),('1808006-03','UNGWMN',385630112321701,'2018-08-06','15:50',NULL,NULL,'paulinkenbrandt','2019-04-15 16:56:52.228920','paulinkenbrandt','2019-04-15 16:56:52.229422',,13052),('1808006-03','UNGWMN',385630112321701,'2018-08-06','15:50',NULL,NULL,'paulinkenbrandt','2019-04-15 16:56:52.228920','paulinkenbrandt','2019-04-15 16:56:52.229422',,13053),('1808006-03','UNGWMN',385630112321701,'2018-08-06','15:50',NULL,NULL,'paulinkenbrandt','2019-04-15 16:56:52.228920','paulinkenbrandt','2019-04-15 16:56:52.229422',,13054),('1808006-03','UNGWMN',

Transaction started...
INSERT INTO UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results(ActivityID,MonitoringLocationID,ResultAnalyticalMethodContext,ResultAnalyticalMethodID,ResultSampleFraction,resultvalue,DetecQuantLimitMeasure,ResultDetecQuantLimitUnit,ResultUnit,AnalysisStartDate,ResultDetecQuantLimitType,ResultDetectionCondition,CharacteristicName,MethodSpeciation,characteristicgroup,inwqx,created_user,last_edited_user,created_date,last_edited_date,resultid,OBJECTID)
VALUES ('1808006-03',385630112321701,'USEPA','310.1','Total',245.0,10.0,'mg/l','mg/l','2018-08-10',NULL,NULL,'Alkalinity',NULL,NULL,0,'paulinkenbrandt','paulinkenbrandt','2019-04-15 16:56:52.228920','2019-04-15 16:56:52.229422','1808006-03-Alkalinity',,14051),('1808006-03',385630112321701,'USEPA','200.8','Total',7.9,2.0,'ug/l','ug/l','2018-08-22',NULL,NULL,'Arsenic',NULL,'Inorganics, Minor, Non-metals',0,'paulinkenbrandt','paulinkenbrandt','2019-04-15 16:56:52.228920','2019-04-15 16:56:52.229422','1808006-03-Arsenic'

0 to 234 complete
Transaction started...
INSERT INTO UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities(ActivityID,ProjectID,MonitoringLocationID,ActivityStartDate,ActivityStartTime,notes,personnel,created_user,created_date,last_edited_user,last_edited_date,OBJECTID)
VALUES ('1808006-03','UNGWMN',385630112321701,'2018-08-06','15:50',NULL,NULL,'paulinkenbrandt','2019-04-15 16:57:15.255639','paulinkenbrandt','2019-04-15 16:57:15.256166',,13051),('1808006-03','UNGWMN',385630112321701,'2018-08-06','15:50',NULL,NULL,'paulinkenbrandt','2019-04-15 16:57:15.255639','paulinkenbrandt','2019-04-15 16:57:15.256166',,13052),('1808006-03','UNGWMN',385630112321701,'2018-08-06','15:50',NULL,NULL,'paulinkenbrandt','2019-04-15 16:57:15.255639','paulinkenbrandt','2019-04-15 16:57:15.256166',,13053),('1808006-03','UNGWMN',385630112321701,'2018-08-06','15:50',NULL,NULL,'paulinkenbrandt','2019-04-15 16:57:15.255639','paulinkenbrandt','2019-04-15 16:57:15.256166',,13054),('1808006-03','UNGWMN',385630112

Transaction started...
INSERT INTO UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results(ActivityID,MonitoringLocationID,ResultAnalyticalMethodContext,ResultAnalyticalMethodID,ResultSampleFraction,resultvalue,DetecQuantLimitMeasure,ResultDetecQuantLimitUnit,ResultUnit,AnalysisStartDate,ResultDetecQuantLimitType,ResultDetectionCondition,CharacteristicName,MethodSpeciation,characteristicgroup,inwqx,created_user,last_edited_user,created_date,last_edited_date,resultid,OBJECTID)
VALUES ('1808006-03',385630112321701,'USEPA','310.1','Total',245.0,10.0,'mg/l','mg/l','2018-08-10',NULL,NULL,'Alkalinity',NULL,NULL,0,'paulinkenbrandt','paulinkenbrandt','2019-04-15 16:57:15.255639','2019-04-15 16:57:15.256166','1808006-03-Alkalinity',,14051),('1808006-03',385630112321701,'USEPA','200.8','Total',7.9,2.0,'ug/l','ug/l','2018-08-22',NULL,NULL,'Arsenic',NULL,'Inorganics, Minor, Non-metals',0,'paulinkenbrandt','paulinkenbrandt','2019-04-15 16:57:15.255639','2019-04-15 16:57:15.256166','1808006-03-Arsenic'

success!


In [ ]:
epa.append_data()

In [ ]:
{'Ammonia as N':['Ammonia','as N'], 
 'Sulfate as SO4':['Sulfate','as SO4'], 
 'Nitrate as N':['Nitrate','as N'], 
 'Nitrite as N':['Nitrite','as N'], 
 'Orthophosphate as P':['Orthophosphate','as P']}


In [ ]:
class ProcessEPASheet(object):

    def __init__(self, file_path, save_path, schema_file_path, user = 'paulinkenbrandt'):
        self.enviro = "C:/Users/paulinkenbrandt/AppData/Roaming/Esri/Desktop10.6/ArcCatalog/Connection to DEFAULT@uggp.agrc.utah.gov.sde"
        arcpy.env.workspace = self.enviro
        self.chem_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results"
        self.activities_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities"
        self.stations_table_name = "UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Locations"
        self.user = user
        self.save_folder = save_path
        self.schema_file_path = schema_file_path

        self.epa_raw_data = pd.read_excel(file_path)
        self.epa_data = None
        self.epa_rename = {'Laboratory':'LaboratoryName',
              'LabNumber':'ActivityID',
              'SampleName':'MonitoringLocationID',
              'Method':'ResultAnalyticalMethodID',
              'Analyte':'CharacteristicName',
              'ReportLimit':'ResultDetecQuantLimitUnit',
              'Result':'ResultValue',
              'AnalyteQual':'ResultQualifier',
              'AnalysisClass':'ResultSampleFraction',
              'ReportLimit':'ResultDetecQuantLimitMeasure'
             }
        
        self.epa_drop = ['Batch','Analysis','Analyst','CASNumber','Elevation','LabQual',
            'Client','ClientMatrix','Dilution','SpkAmt','UpperLimit','Recovery',
           'Surrogate','LowerLimit','Latitude','Longitude','SampleID','ProjectNumber',
            'Sampled','Analyzed','PrepMethod','Prepped','Project']
        
    def hasless(self, df):
        if '<' in str(df['ResultValue']):
            df['ResultValue'] = None
            df['ResultDetectionCondition'] = 'Below Reporting Limit'
            df['ResultDetectQuantLimitType'] = 'Lower Reporting Limit'
        elif '>' in str(df['ResultValue']):
            df['ResultValue'] = None
            df['ResultDetectionCondition'] = 'Above Reporting Limit'
            df['ResultDetectQuantLimitType'] = 'Upper Reporting Limit'
        else:
            df['ResultValue'] = pd.to_numeric(df['ResultValue'])
        return df

    def resqual(self, x):
        if pd.isna(x[1]) and x[0] == 'Below Reporting Limit':
            return 'BRL'
        elif pd.notnull(x[1]):
            return x[1]
        else:
            return None
    
    def filtmeth(self, x):
        if "EPA" in x:
            x = x.split(' ')[1]
        elif '/' in x:
            x = x.split('/')[0]
        else:
            x = x
        return x
    
    def save_it(self, savefolder):
        self.epa_data.to_csv("{:}/epa_sheet_to_sde_{:%Y%m%d%M%H}.csv".format(savefolder, pd.datetime.today()))
    
    def run_calcs(self):
        epa_raw_data = self.epa_raw_data
        epa_raw_data = epa_raw_data.rename(columns=self.epa_rename)
        epa_raw_data['ResultSampleFraction'] = epa_raw_data['ResultSampleFraction'].apply(lambda x: 'Total' if 'WET' else x, 1)
        epa_raw_data = epa_raw_data.apply(lambda x: self.hasless(x),1)
        epa_raw_data['ResultAnalyticalMethodID'] = epa_raw_data['ResultAnalyticalMethodID'].apply(lambda x: self.filtmeth(x), 1)
        epa_raw_data['ResultAnalyticalMethodContext'] = 'USEPA'
        epa_raw_data['ProjectID'] = 'UNGWMN'
        epa_raw_data['ResultQualifier'] = epa_raw_data[['ResultDetectionCondition',
                                                        'ResultQualifier']].apply(lambda x: self.resqual(x),1)
        epa_raw_data['Sample Date'] = epa_raw_data['Sampled'].apply(lambda x: "{:%Y-%m-%d}".format(x),1)
        epa_raw_data['Sample Time'] = epa_raw_data['Sampled'].apply(lambda x: "{:%H:%M}".format(x),1)
        epa_raw_data['AnalysisStartDate'] = epa_raw_data['Analyzed'].apply(lambda x: "{:%Y-%m-%d}".format(x),1)
        epa_data = epa_raw_data.drop(epa_drop,axis=1)
        self.epa_data = epa_data
        self.save_it(self.save_folder)
        return epa_data
        
    def append_data(self):
        epa_chem = self.run_calcs()
        sdeact = table_to_pandas_dataframe(self.activities_table_name, field_names=['MonitoringLocationID', 'ActivityID'])
        sdechem = table_to_pandas_dataframe(self.chem_table_name, field_names=['MonitoringLocationID', 'ActivityID'])

        epa_chem['created_user'] = self.user
        epa_chem['last_edited_user'] = self.user
        epa_chem['created_date'] = pd.datetime.today()
        epa_chem['last_edited_date'] = pd.datetime.today()

        try:
            df = epa_chem[~epa_chem['ActivityID'].isin(sdeact['ActivityID'])]
            fieldnames = ['ActivityID', 'ProjectID', 'MonitoringLocationID', 'ActivityStartDate',
                          'ActivityStartTime', 'notes', 'personnel', 'created_user', 'created_date', 'last_edited_user',
                          'last_edited_date']

            for i in range(0, len(df), 500):
                j = i + 500
                if j > len(df):
                    j = len(df)
                subset = df.iloc[i:j]
                print("{:} to {:} complete".format(i, j))
                edit_table(subset, self.activities_table_name, fieldnames=fieldnames, enviro = self.enviro)
                print('success!')
        except Exception as err:
            print(err)
            print('fail!')
            pass

        try:
            df = epa_chem[~epa_chem['ActivityID'].isin(sdechem['ActivityID'])]
            fieldnames = ['ActivityID', 'MonitoringLocationID', 'ResultAnalyticalMethodContext', 'ResultAnalyticalMethodID',
                          'ResultSampleFraction',
                          'resultvalue', 'ResultDetecQuantLimitMeasure', 'ResultDetecQuantLimitUnit', 'ResultUnit',
                          'AnalysisStartDate', 'ResultDetecQuantLimitType', 'ResultDetectionCondition',
                          'CharacteristicName',
                          'MethodSpeciation', 'characteristicgroup', 'ResultValueType', 'ResultStatusID',
                          'inwqx', 'created_user', 'last_edited_user', 'created_date', 'last_edited_date']

            for i in range(0, len(df), 500):
                j = i + 500
                if j > len(df):
                    j = len(df)
                subset = df.iloc[i:j]
                print("{:} to {:} complete".format(i, j))
                edit_table(subset, self.chem_table_name, fieldnames = fieldnames, enviro = self.enviro)
                print('success!')
        except Exception as err:
            print(err)
            print('fail!')
            pass

In [ ]:
epa_file = "G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/EPAdata_results/2018/1807003 FINAL EPA Excel 12 Sep 18 1357.xls"

epa_raw_data = pd.read_excel(epa_file)

epa_rename = {'Laboratory':'LaboratoryName',
              'LabNumber':'ActivityID',
              'SampleName':'MonitoringLocationID',
              'Method':'ResultAnalyticalMethodID',
              'Analyte':'CharacteristicName',
              'ReportLimit':'ResultDetecQuantLimitUnit',
              'Result':'ResultValue',
              'AnalyteQual':'ResultQualifier',
              'AnalysisClass':'ResultSampleFraction',
              'ReportLimit':'ResultDetecQuantLimitMeasure'
             }

epa_raw_data = epa_raw_data.rename(columns=epa_rename)

epa_raw_data['ResultSampleFraction'] = epa_raw_data['ResultSampleFraction'].apply(lambda x: 'Total' if 'WET' else x, 1)

def hasless(df):
    if '<' in str(df['ResultValue']):
        df['ResultValue'] = None
        df['ResultDetectionCondition'] = 'Below Reporting Limit'
        df['ResultDetectQuantLimitType'] = 'Lower Reporting Limit'
    elif '>' in str(df['ResultValue']):
        df['ResultValue'] = None
        df['ResultDetectionCondition'] = 'Above Reporting Limit'
        df['ResultDetectQuantLimitType'] = 'Upper Reporting Limit'
    else:
        df['ResultValue'] = pd.to_numeric(df['ResultValue'])
    return df

epa_raw_data = epa_raw_data.apply(lambda x: hasless(x),1)


def resqual(x):
    if pd.isna(x[1]) and x[0] == 'Below Reporting Limit':
        return 'BRL'
    elif pd.notnull(x[1]):
        return x[1]
    else:
        return None
    

def filtmeth(x):
    if "EPA" in x:
        x = x.split(' ')[1]
    elif '/' in x:
        x = x.split('/')[0]
    else:
        x = x
    return x


epa_raw_data['ResultAnalyticalMethodID'] = epa_raw_data['ResultAnalyticalMethodID'].apply(lambda x: filtmeth(x), 1)
epa_raw_data['ResultAnalyticalMethodContext'] = 'USEPA'
epa_raw_data['ProjectID'] = 'UNGWMN'
epa_raw_data['ResultQualifier'] = epa_raw_data[['ResultDetectionCondition',
                                                'ResultQualifier']].apply(lambda x: resqual(x),1)

epa_raw_data['Sample Date'] = epa_raw_data['Sampled'].apply(lambda x: "{:%Y-%m-%d}".format(x),1)
epa_raw_data['Sample Time'] = epa_raw_data['Sampled'].apply(lambda x: "{:%H:%M}".format(x),1)
epa_raw_data['AnalysisStartDate'] = epa_raw_data['Analyzed'].apply(lambda x: "{:%Y-%m-%d}".format(x),1)

epa_drop = ['Batch','Analysis','Analyst','CASNumber','Elevation','LabQual',
            'Client','ClientMatrix','Dilution','SpkAmt','UpperLimit','Recovery',
           'Surrogate','LowerLimit','Latitude','Longitude','SampleID','ProjectNumber',
            'Sampled','Analyzed','PrepMethod','Prepped','Project']
epa_data = epa_raw_data.drop(epa_drop,axis=1)
epa_data

In [ ]:
sdechem.columns

In [ ]:
sdeact.columns

In [ ]:
epa_raw_data['ResultQualifier']

## State Lab Chemistry Import to SDE

### Compile Old State Lab Data

In [ ]:
slfiles = "G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/StateLab/"

old_to_new = {'Sample#Number':'Sample Number',
              'Sample#Date':'Date Collected',
              'Sample#Time':'Time Collected',
              'Analysis#Date':'Analysis Date',
              'CAS#Number':'CAS Number',
              'Agency#Bill#Code':'Agency Bill Code',
              'Batch#Number':'Batch Number',
              'Comment#Text':'Result Comment',
              'Chain#of#Custody#Ind.':'Chain of Custody',
              'Cost#Code':'Cost Code',
              'Dilution#Factor':'Dilution Factor',
              'Lower#Report#Limit':'Lower Report Limit',
              'Matrix#Description':'Matrix Description',
              'Method#Agency':'Method Agency',
              'Method#Description':'Method Description',
              'Method#Detect#Limit':'Method Detect Limit',
              'Param#Description':'Param Description',
              'Problem#Identifier':'Problem Identifier',
              'Method#ID':'Method ID',
              'Sample#Comment':'Sample Comment',
              'Sample#Description':'Sample Description',
              'Result#Value':'Result Value',
              'Sample#Recieved#Date':'Sample Received Date',
              'Replicate#Number':'Replicate Number',
              'Result#Code':'Result#Code',
              'Station#ID':'Station ID',
              'Sample#Type':'Sample Type',
              'Trip#ID':'Trip ID'}

oldfields = ['%#Confidence#Limit', 'Confidence#Limit', 'Comment#Number', 'Comment#Number.1', 'Comment#Text.1',
             'Lab#Code', 'Preparation#Date', 'Source#Code',  'Billing#Code','Matrix#Number',
             'Upper#Quant#Limit','Result#Code','Method#Number','Param#Number']


stlbnew = {}
stlbold = {}

for file in glob.glob(slfiles+"*.txt"):
    
    shrtnm = os.path.splitext(os.path.basename(file))[0]
    if "UTGS_EDD" in shrtnm: 
        stlbnew[shrtnm] = pd.read_csv(file, sep='\t', error_bad_lines=False)
        samecols = {'Collected By':'Collector',
                    'Analyte CAS Number':'CAS Number',
                    'Method Number':'Method ID',
                    'Method Reported Name':'Param Description',
                   'Entry Qualifier':'Problem Identifier',
                    'Result Flags':'Result Code',
                   'Result Entry':'Result Value',
                    'Sample Detection Limit':'Sample Detect Limit',
                    'Date Analyzed':'Analysis Date',
                   'Method Detection Limit':'Method Detect Limit','Project Comments':'Project Comment',
                   'Sample Comments':'Sample Comment','Test Comments':'Test Comment',
                   'Result Comments':'Result Comment','Sample Reporting Limit':'Sample Report Limit',
                   'Result Dilution Factor':'Dilution Factor','Date Recieved':'Sample Received Date',
                   'Specimen Source':'Matrix Description','Customer Internal Code':'Cost Code',
                   'Collecting Agency':'Agency Bill Code',
                   'Result Units':'Units'}
        stlbnew[shrtnm] = stlbnew[shrtnm].rename(columns=samecols)
    else:
        stlbold[shrtnm] = pd.read_csv(file, sep='\t', error_bad_lines=False)
        stlbold[shrtnm] = stlbold[shrtnm].rename(columns=old_to_new)
new_st_data = pd.concat(stlbnew).reset_index().drop(['level_0','level_1'],axis=1)
old_st_data = pd.concat(stlbold).reset_index().drop(['level_0','level_1'],axis=1)
st_data = pd.concat([new_st_data,old_st_data])
st_data = st_data.drop(oldfields, axis=1)


st_data.columns

In [ ]:
st_data.head()

### Designate the file with the id matches.

In [ ]:
lab_based_chem = "https://cdx.epa.gov/WQXWeb/ImportConfigurationDetail.aspx?impcfg_uid=5926"

field_chem = "https://cdx.epa.gov/WQXWeb/ImportConfigurationDetail.aspx?mode=import&impcfg_uid=5931"

In [ ]:
match_file = "G:/My Drive/Python/Pycharm/wqxsde/examples/id_match.csv"
file_path = "G:/My Drive/Python/Pycharm/wqxsde/examples/UTGS_EDD_20190304.txt"
file_path2 = "G:/Team Drives/UGS_Groundwater/Projects/NGWMN/Chemistry Data/StateLab/UTGS_EDD_20170907.txt"
schema_file_path = "G:/My Drive/Python/Pycharm/wqxsde/examples/Domains.xlsx"

In [ ]:
wqx_chem_table, sde_chem_table = compare_sde_wqx(wqx_results_filename,enviro,chem_table_name,table_type="chem")
sde_chem_import = sde_chem_table[sde_chem_table['inwqx']==0]

### Process the state lab text file to import into database

In [ ]:
state = wqxsde.ProcessStateLabText(file_path,"G:/My Drive/WORK/Groundwater Chemistry",
                                   match_file, schema_file_path)


In [ ]:
state = wqxsde.ProcessStateLabText(file_path,"")

In [ ]:
state_lab_chem = state.run_calcs()

In [ ]:
state_lab_chem['resultid'] = state_lab_chem[['ActivityID','CharacteristicName']].apply(lambda x: str(x[0])+ "-" +x[1],1) 

In [ ]:
state_lab_chem.to_csv("G:/My Drive/WORK/Groundwater Chemistry/crosscheck.csv")

### Import SDE Data

In [ ]:
state.append_data()

## Pull SDE Data and Prep for WQX import

In [ ]:
chemfields = ['ActivityID', 'ResultDetectionCondition', 'CharacteristicName', 'MethodSpeciation',
              'ResultSampleFraction', 'ResultValue', 'ResultUnit', 'ResultQualifier', 
              'ResultStatusID', 'ResultValueType', 'ResultAnalyticalMethodID', 
              'ResultAnalyticalMethodContext', 'LaboratoryName',
              'AnalysisStartDate', 'ResultDetecQuantLimitType',
              'ResultDetecQuantLimitMeasure', 'ResultDetecQuantLimitUnit']

activfields = ['ActivityID', 'ProjectID', 'MonitoringLocationID', 
               'ActivityStartDate', 'ActivityStartTime','temperature_c','conduct_uscm','pH']

sdeact = wqxsde.table_to_pandas_dataframe(sde.activities_table, field_names = activfields)
sdechem = wqxsde.table_to_pandas_dataframe(sde.chemistry_table, field_names = chemfields, query = 'inwqx = 0' )

In [ ]:
sdechem.head()

In [ ]:
onlineacts

In [ ]:
afields = ['ActivityID', 'ProjectID', 'MonitoringLocationID', 
               'ActivityStartDate', 'ActivityStartTime']

actmatch = onlineacts[afields].set_index('ActivityID').to_dict()

In [ ]:
sdefieldact.head()

In [ ]:
sdefieldact = onlineacts[['ActivityID','temperature_c','conduct_uscm','pH']].set_index('ActivityID').stack().reset_index()
sdefieldact['ActivityID'] = sdefieldact['ActivityID'].apply(lambda x: str(x)+'-FM',1)
sdefieldact.columns = ['ActivityID','CharacteristicName','ResultValue']
fcharnms = {'temperature_c':['Temperature, water','deg C'],'conduct_uscm':['Specific conductance','uS/cm'],'pH':['pH','None']}

def funitsandchar(df):
    fcharnms = {'temperature_c':['Temperature, water','deg C'],
                'conduct_uscm':['Specific conductance','uS/cm'],
                'pH':['pH','None']}
    df['ResultUnit'] = fcharnms.get(df['CharacteristicName'])[1]
    df['CharacteristicName'] = fcharnms.get(df['CharacteristicName'])[0]
    return df

sdefieldact = sdefieldact.apply(lambda x: funitsandchar(x),1)

In [ ]:
actmatch.keys()

In [ ]:
sdefieldact['ActivityStartDate'] = sdefieldact['ActivityID'].apply(lambda)

In [ ]:
sdeactchem = pd.merge(sdechem, sdeact, how = 'inner', on = 'ActivityID')

In [ ]:
sdeactchem['Activity Media Name'] = 'Water'
sdeactchem['Activity Start Time Zone'] = 'MDT'
sdeactchem['Activity Type'] = sdeactchem['ActivityID'].apply(lambda x: 'Field Msr/Obs' if '-FM' in x else 'Sample-Routine', 1)
sdeactchem['Sample Collection Equipment Name'] = 'Water Bottle'
sdeactchem['Sample Collection Method ID'] = 'GRAB'
sdeactchem['ResultStatusID'] = 'Final'
sdeactchem['ResultValueType'] = 'Actual'

In [ ]:
fieldorder = ['ActivityID', 'Activity Media Name', 'ActivityStartDate', 'ActivityStartTime',
              'Activity Start Time Zone', 'Activity Type', 'AnalysisStartDate', 
              'CharacteristicName', 'LaboratoryName', 'MethodSpeciation',
              'MonitoringLocationID', 'ProjectID', 'ResultAnalyticalMethodContext',
              'ResultAnalyticalMethodID', 'ResultDetectionCondition',
              'ResultDetecQuantLimitMeasure', 'ResultDetecQuantLimitType',
              'ResultDetecQuantLimitUnit', 'ResultQualifier', 'ResultSampleFraction', 
              'ResultStatusID', 'ResultUnit',  'ResultValue', 'ResultValueType',
              'Sample Collection Equipment Name','Sample Collection Method ID']


sdeactchemimp = sdeactchem[fieldorder].replace(to_replace='None',value=None)
#sdeactchemimp = sdeactchemimp[sdeactchemimp['Activity Type']=='Sample-Routine']
sdeactchemimp = sdeactchemimp[pd.notna(sdeactchemimp['ProjectID'])]

In [ ]:
sdeactchemimp

In [ ]:
sdeactchemimp['ProjectID'].unique()

In [ ]:
save_dir = 'G:/My Drive/Python/Pycharm/wqxsde'
sdeactchemimp.to_csv(save_dir + "/chem_{:%Y%m%d%H}.csv".format(pd.datetime.today()), index=False)

In [ ]:
activities = 

In [ ]:
stations[['LocationName','AltLocationID','Offset','BaroLoggerType']][stations['AltLocationID']>1000].to_clipboard()

~~~~sql
CREATE VIEW UGGP.UGGPADMIN.UGS_NGWMN_View_Activities_and_Results
AS
SELECT 
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityStartDate,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityStartTime,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ProjectID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.MonitoringLocationID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetectionCondition,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.CharacteristicName,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.MethodSpeciation,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultSampleFraction,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultValue,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultUnit,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultQualifier,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultStatusID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultValueType,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultAnalyticalMethodID,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultAnalyticalMethodContext,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.LaboratoryName,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.AnalysisStartDate,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetecQuantLimitType,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetecQuantLimitMeasure,
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ResultDetecQuantLimitUnit

FROM
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results 
JOIN
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities ON
UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Activities.ActivityID = UGGP.UGGPADMIN.UGS_NGWMN_Monitoring_Phy_Chem_Results.ActivityID
~~~~
